## Deep Learning Framework Power Scores 2018
## By Jeff Hale

### See [this Medium article](https://towardsdatascience.com/deep-learning-framework-power-scores-2018-23607ddf297a) for a discussion of the state of Python deep learning frameworks in 2018 featuring these charts.

I'm going to use plotly and pandas to make interactive visuals for this project.

# Please upvote this Kaggle kernel if you find it helpful.

In [4]:
# import the usual frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler

import os
print(os.listdir("../input"))
    
# import plotly 
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl 

# define color scale https://plot.ly/ipython-notebooks/color-scales/
cs = cl.scales['7']['qual']['Dark2']     

# configure things
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format  
pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline

['dsframeworks.csv']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


List package versions for reproducibility.

In [5]:
#!pip list

Read in the data from the csv. The Google sheet that holds the data is available [here](https://docs.google.com/spreadsheets/d/1mYfHMZfuXGpZ0ggBVDot3SJMU-VsCsEGceEL8xd1QBo/edit?usp=sharing).

In [6]:
new_col_names = ['framework','indeed', 'monster', 'simply', 'linkedin', 'angel', 
                 'usage', 'search', 'medium', 'books', 'arxiv', 'stars', 
                 'watchers', 'forks', 'contribs',
                ]

df = pd.read_csv('../input/dsframeworks.csv', 
                 skiprows=4,
                 header=None, 
                 nrows=7, 
                 thousands=',',
                 index_col=0,
                 names=new_col_names,
                 usecols=new_col_names,
                )
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs
framework,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29.90%,78,45300,202,201,109576,8334,67551,1642
Keras,684,364,449,695,177,22.20%,38,48200,79,36,33558,1847,12658,719
PyTorch,486,309,428,665,120,6.40%,30,6410,18,42,18716,952,4474,760
Theano,356,316,279,508,95,4.90%,1,1670,17,36,8477,585,2447,328
MXNET,266,154,200,298,29,1.50%,3,2010,32,26,15200,1170,5498,587
CNTK,126,96,97,160,12,3.00%,1,675,1,13,15106,1368,4029,189
FastAI,0,0,0,0,0,0.00%,2,4480,0,1,7268,432,2647,195


Cool. We used the read_csv parameters to give us just what we wanted.

## Basic Data Exploration
Let's see what the data look like.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, TensorFlow to FastAI
Data columns (total 14 columns):
indeed      7 non-null int64
monster     7 non-null int64
simply      7 non-null int64
linkedin    7 non-null int64
angel       7 non-null int64
usage       7 non-null object
search      7 non-null int64
medium      7 non-null int64
books       7 non-null int64
arxiv       7 non-null int64
stars       7 non-null int64
watchers    7 non-null int64
forks       7 non-null int64
contribs    7 non-null int64
dtypes: int64(13), object(1)
memory usage: 840.0+ bytes


In [8]:
df.describe()

,indeed,monster,simply,linkedin,angel,search,medium,books,arxiv,stars,watchers,forks,contribs
count,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00
mean,571.00,356.00,433.57,705.14,140.71,21.86,"15,535.00",49.86,50.71,"29,700.14","2,098.29","14,186.29",631.43
std,702.07,417.06,532.11,878.37,192.28,29.15,"21,425.92",72.21,67.82,"36,268.80","2,790.54","23,784.03",504.27
min,0.00,0.00,0.00,0.00,0.00,1.00,675.00,0.00,1.00,"7,268.00",432.00,"2,447.00",189.00
25%,196.00,125.00,148.50,229.00,20.50,1.50,"1,840.00",9.00,19.50,"11,791.50",768.50,"3,338.00",261.50
50%,356.00,309.00,279.00,508.00,95.00,3.00,"4,480.00",18.00,36.00,"15,200.00","1,170.00","4,474.00",587.00
75%,585.00,340.00,438.50,680.00,148.50,34.00,"25,855.00",55.50,39.00,"26,137.00","1,607.50","9,078.00",739.50
max,"2,079.00","1,253.00","1,582.00","2,610.00",552.00,78.00,"48,200.00",202.00,201.00,"109,576.00","8,334.00","67,551.00","1,642.00"


Looks like pandas read the usage column as a string because of it's percent sign. Let's make that a decimal.

In [9]:
df['usage'] = pd.to_numeric(df['usage'].str.strip('%'))
df['usage'] = df['usage'].astype(int)
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs
framework,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29,78,45300,202,201,109576,8334,67551,1642
Keras,684,364,449,695,177,22,38,48200,79,36,33558,1847,12658,719
PyTorch,486,309,428,665,120,6,30,6410,18,42,18716,952,4474,760
Theano,356,316,279,508,95,4,1,1670,17,36,8477,585,2447,328
MXNET,266,154,200,298,29,1,3,2010,32,26,15200,1170,5498,587
CNTK,126,96,97,160,12,3,1,675,1,13,15106,1368,4029,189
FastAI,0,0,0,0,0,0,2,4480,0,1,7268,432,2647,195


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, TensorFlow to FastAI
Data columns (total 14 columns):
indeed      7 non-null int64
monster     7 non-null int64
simply      7 non-null int64
linkedin    7 non-null int64
angel       7 non-null int64
usage       7 non-null int64
search      7 non-null int64
medium      7 non-null int64
books       7 non-null int64
arxiv       7 non-null int64
stars       7 non-null int64
watchers    7 non-null int64
forks       7 non-null int64
contribs    7 non-null int64
dtypes: int64(14)
memory usage: 840.0+ bytes


All ints! Great!

# Plotly
Let's make interactive plots with plotly for each popularity category.

## Online Job Listings
I looked at how many times each framework appeared in searches on job listing websites. For more discussion see the Medium Article the accompanies this notebook here.

In [11]:
# sum groupby for the hiring columns
df['hiring'] = df['indeed'] + df['monster'] + df['linkedin'] + df['simply'] + df['angel']

In [12]:
df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring
framework,,,,,,,,,,,,,,,
TensorFlow,2079,1253,1582,2610,552,29,78,45300,202,201,109576,8334,67551,1642,8076
Keras,684,364,449,695,177,22,38,48200,79,36,33558,1847,12658,719,2369
PyTorch,486,309,428,665,120,6,30,6410,18,42,18716,952,4474,760,2008
Theano,356,316,279,508,95,4,1,1670,17,36,8477,585,2447,328,1554
MXNET,266,154,200,298,29,1,3,2010,32,26,15200,1170,5498,587,947
CNTK,126,96,97,160,12,3,1,675,1,13,15106,1368,4029,189,491
FastAI,0,0,0,0,0,0,2,4480,0,1,7268,432,2647,195,0


In [13]:
data = [go.Bar(
    x=df.index,
    y=df.hiring,
    marker=dict(color=cs),
    )
]

layout = {'title': 'Online Job Listings',
          'xaxis': {'title': 'Framework'},
          'yaxis': {'title': "Quantity"},
         }

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

That's just the aggregate listings. Let's plot the job listing mentions for each website in a stacked bar chart. This will take multiple traces.

In [14]:
y_indeed = df['indeed']
y_monster = df['monster']
y_simply = df['simply']
y_linkedin = df['linkedin']
y_angel = df['angel']

In [15]:
indeed = go.Bar(x=df.index, y=y_indeed, name = 'Indeed')
simply = go.Bar(x=df.index, y=y_simply, name='Simply Hired')
monster = go.Bar(x=df.index, y=y_monster, name='Monster')
linked = go.Bar(x=df.index, y=y_linkedin, name='LinkedIn')
angel = go.Bar(x=df.index, y=y_angel, name='Angel List')

data = [linked, indeed, simply, monster, angel]
layout = go.Layout(
    barmode='stack',
    title='Online Job Listings',
    xaxis={'title': 'Framework'},
    yaxis={'title': 'Mentions', 'separatethousands': True},
    colorway=cs,
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

Cool. Now let's see how this data looks with grouped bars instead of stacked bars by changing the barmode to "group".

In [16]:
indeed = go.Bar(x=df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=df.index, y=y_simply, name="Simply Hired")
monster = go.Bar(x=df.index, y=y_monster, name="Monster")
linked = go.Bar(x=df.index, y=y_linkedin, name="LinkedIn")
angel = go.Bar(x=df.index, y=y_angel, name='Angel List')

data = [linked, indeed, simply, monster, angel]
layout = go.Layout(
    barmode='group',
    title="Online Job Listings",
    xaxis={'title': 'Framework'},
    yaxis={'title': "Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## KDnuggets Usage Survey
Let's look at usage as reported in KDnuggets 2018 survey.

In [17]:
# Make sure you have colorlover imported as cl for color scale
df['usage'] = df['usage'] / 100

## Google Search Volume

In [18]:
data = [
    go.Bar(
        x=df.index, 
        y=df['usage'],
        marker=dict(color=cs)  
    )
]
    
layout = {
    'title': 'KDnuggets Usage Survey',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "% Respondents Used in Past Year", 'tickformat': '.0%'},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [19]:
data = [
    go.Bar(
        x = df.index, 
        y = df['search'],
        marker = dict(color=cs),  
    )
]
    
layout = {
    'title': 'Google Search Volume',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Relative Search Volume"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Medium Articles

In [20]:
# Make sure you have colorlover imported as cl for color scale
# cs is defined in first cell

data = [
    go.Bar(
        x=df.index, 
        y=df['medium'],
        marker=dict(color=cs) ,
    )
]
    
layout = {
    'title': 'Medium Articles',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Articles"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Amazon Books

In [21]:
data = [
    go.Bar(
        x=df.index, 
        y=df['books'],
        marker=dict(color=cs),           
    )
]
    
layout = {
    'title': 'Amazon Books',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Books"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## ArXiv Articles

In [22]:
data = [
    go.Bar(
        x=df.index, 
        y=df['arxiv'],
        marker=dict(color=cs),           
    )
]

layout = {
    'title': 'ArXiv Articles',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Articles"},
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# GitHub Activity
Let's make another stacked bar chart of the four GitHub categories.

In [23]:
y_stars = df['stars']
y_watchers = df['watchers']
y_forks = df['forks']
y_contribs = df['contribs']

stars = go.Bar(x = df.index, y=y_stars, name="Stars")
watchers = go.Bar(x=df.index, y=y_watchers, name="Watchers")
forks = go.Bar(x=df.index, y=y_forks, name="Forks")
contribs = go.Bar(x=df.index, y=y_contribs, name="Contributors")


data = [stars, watchers, forks, contribs]
layout = go.Layout(barmode='stack', 
    title="GitHub Activity",
    xaxis={'title': 'Framework'},
    yaxis={
        'title': "Quantity",
        'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

This configuration doesn't make the most sense, because there are going to be way more stars than contributors. It's not an apples to apples comparison. Let's try four subplots instead.

In [24]:
trace1 = go.Bar(
    x=df.index,
    y=df['stars'],
    name='Stars',
    marker=dict(color=cs),
)
trace2 = go.Bar(
    x=df.index,
    y=df['forks'],
    name ="Forks",
    marker=dict(color=cs)
)
trace3 = go.Bar(
    x=df.index,
    y=df['watchers'],
    name='Watchers',
    marker=dict(color=cs)
)
trace4 = go.Bar(
    x=df.index,
    y=df['contribs'],
    name='Contributors',
    marker=dict(color=cs),
)

fig = tls.make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=(
        'Stars', 
        'Forks',
        'Watchers',
        'Contributors',
    )
)

fig['layout']['yaxis3'].update(separatethousands = True)
fig['layout']['yaxis4'].update(separatethousands = True)
fig['layout']['yaxis2'].update(tickformat = ',k', separatethousands = True)
fig['layout']['yaxis1'].update(tickformat = ',k', separatethousands = True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title = 'GitHub Activity', showlegend = False)
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



This presentation shows the information in a more comprehensible and appropriate format.

# Scale and Aggregate for Power Scores
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, TensorFlow to FastAI
Data columns (total 15 columns):
indeed      7 non-null int64
monster     7 non-null int64
simply      7 non-null int64
linkedin    7 non-null int64
angel       7 non-null int64
usage       7 non-null float64
search      7 non-null int64
medium      7 non-null int64
books       7 non-null int64
arxiv       7 non-null int64
stars       7 non-null int64
watchers    7 non-null int64
forks       7 non-null int64
contribs    7 non-null int64
hiring      7 non-null int64
dtypes: float64(1), int64(14)
memory usage: 896.0+ bytes


In [26]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df), 
    columns = df.columns,
    index = df.index)    

In [27]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring
framework,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.12,0.09,0.20,0.11,0.07,0.03,0.39,0.25
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.02,0.08,0.17,0.01,0.02,0.00,0.10,0.19
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.03,0.16,0.12,0.08,0.09,0.05,0.27,0.12
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.00,0.00,0.06,0.08,0.12,0.02,0.00,0.06
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Scaled Online Job Listings
Let's combine the scaled online job listing columns into a new column.

In [28]:
scaled_df['hiring_score'] = scaled_df[['indeed', 'monster', 'simply', 'linkedin', 'angel']].mean(axis=1)

In [29]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring,hiring_score
framework,,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29,0.30
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.12,0.09,0.20,0.11,0.07,0.03,0.39,0.25,0.24
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.02,0.08,0.17,0.01,0.02,0.00,0.10,0.19,0.19
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.03,0.16,0.12,0.08,0.09,0.05,0.27,0.12,0.11
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.00,0.00,0.06,0.08,0.12,0.02,0.00,0.06,0.06
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Now we have a hiring score.

### Scaled GitHub Activity

Let's combine the scaled GitHub columns into a new column.

In [30]:
scaled_df['github_score'] = scaled_df[['stars', 'watchers', 'forks', 'contribs']].mean(axis=1)

In [31]:
scaled_df

,indeed,monster,simply,linkedin,angel,usage,search,medium,books,arxiv,stars,watchers,forks,contribs,hiring,hiring_score,github_score
framework,,,,,,,,,,,,,,,,,
TensorFlow,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.94,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Keras,0.33,0.29,0.28,0.27,0.32,0.76,0.48,1.00,0.39,0.17,0.26,0.18,0.16,0.36,0.29,0.30,0.24
PyTorch,0.23,0.25,0.27,0.25,0.22,0.21,0.38,0.12,0.09,0.20,0.11,0.07,0.03,0.39,0.25,0.24,0.15
Theano,0.17,0.25,0.18,0.19,0.17,0.14,0.00,0.02,0.08,0.17,0.01,0.02,0.00,0.10,0.19,0.19,0.03
MXNET,0.13,0.12,0.13,0.11,0.05,0.03,0.03,0.03,0.16,0.12,0.08,0.09,0.05,0.27,0.12,0.11,0.12
CNTK,0.06,0.08,0.06,0.06,0.02,0.10,0.00,0.00,0.00,0.06,0.08,0.12,0.02,0.00,0.06,0.06,0.05
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Now we have all our aggregate columns and are ready to turn to the weights.

## Weights

Let's make a pie chart of weights by category.

In [32]:
weights = {'Online Job Listings ': .3,
           'KDnuggets Usage Survey': .2,
           'GitHub Activity': .1,
           'Google Search Volume': .1,
           'Medium Articles': .1,
           'Amazon Books': .1,
           'ArXiv Articles': .1 }

In [33]:
# changing colors because we want to show these aren't the frameworks
weight_colors = cl.scales['7']['qual']['Set1'] 

common_props = dict(
    labels = list(weights.keys()),
    values = list(weights.values()),
    textfont=dict(size=16),
    marker=dict(colors=weight_colors),
    hoverinfo='none',
    showlegend=False,
)

trace1 = go.Pie(
    **common_props,
    textinfo='label',
    textposition='outside',
)

trace2 = go.Pie(
    **common_props,
    textinfo='percent',
    textposition='inside',
)

layout = go.Layout(title = 'Weights by Category')

fig = go.Figure([trace1, trace2], layout=layout)
py.iplot(fig)

## Weight the Categories

In [34]:
scaled_df['w_hiring'] = scaled_df['hiring_score'] * .3
scaled_df['w_usage'] = scaled_df['usage'] * .2
scaled_df['w_github'] = scaled_df['github_score'] * .1
scaled_df['w_search'] = scaled_df['search'] * .1
scaled_df['w_arxiv'] = scaled_df['arxiv'] * .1
scaled_df['w_books'] = scaled_df['books'] * .1
scaled_df['w_medium'] = scaled_df['medium'] * .1

In [35]:
weight_list = ['w_hiring', 'w_usage', 'w_github', 'w_search', 'w_arxiv', 'w_books', 'w_medium']
scaled_df = scaled_df[weight_list]
scaled_df

,w_hiring,w_usage,w_github,w_search,w_arxiv,w_books,w_medium
framework,,,,,,,
TensorFlow,0.30,0.20,0.10,0.10,0.10,0.10,0.09
Keras,0.09,0.15,0.02,0.05,0.02,0.04,0.10
PyTorch,0.07,0.04,0.02,0.04,0.02,0.01,0.01
Theano,0.06,0.03,0.00,0.00,0.02,0.01,0.00
MXNET,0.03,0.01,0.01,0.00,0.01,0.02,0.00
CNTK,0.02,0.02,0.01,0.00,0.01,0.00,0.00
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01


## Power Scores
Let's make the power score column by summing the seven category scores.

In [36]:
scaled_df['ps'] = scaled_df[weight_list].sum(axis = 1)
scaled_df

,w_hiring,w_usage,w_github,w_search,w_arxiv,w_books,w_medium,ps
framework,,,,,,,,
TensorFlow,0.30,0.20,0.10,0.10,0.10,0.10,0.09,0.99
Keras,0.09,0.15,0.02,0.05,0.02,0.04,0.10,0.47
PyTorch,0.07,0.04,0.02,0.04,0.02,0.01,0.01,0.21
Theano,0.06,0.03,0.00,0.00,0.02,0.01,0.00,0.12
MXNET,0.03,0.01,0.01,0.00,0.01,0.02,0.00,0.09
CNTK,0.02,0.02,0.01,0.00,0.01,0.00,0.00,0.05
FastAI,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01


Let's clean things up for publication

In [37]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['Job Search Listings', 'Usage Survey', 'Search Volume', 'Medium Articles', 'Amazon Books', 'ArXiv Articles', 'GitHub Activity', 'Power Score']
p_s_df.rename_axis('Framework', inplace = True)
p_s_df

,Job Search Listings,Usage Survey,Search Volume,Medium Articles,Amazon Books,ArXiv Articles,GitHub Activity,Power Score
Framework,,,,,,,,
TensorFlow,30.00,20.00,10.00,10.00,10.00,10.00,9.39,99.39
Keras,8.94,15.17,2.39,4.81,1.75,3.91,10.00,46.97
PyTorch,7.34,4.14,1.50,3.77,2.05,0.89,1.21,20.90
Theano,5.80,2.76,0.32,0.00,1.75,0.84,0.21,11.68
MXNET,3.26,0.69,1.23,0.26,1.25,1.58,0.28,8.56
CNTK,1.69,2.07,0.55,0.00,0.60,0.05,0.00,4.96
FastAI,0.00,0.00,0.02,0.13,0.00,0.00,0.80,0.95


Let's make a bar chart of the power scores.

In [38]:
data = [
    go.Bar(
        x=scaled_df.index,          # you can pass plotly the axis
        y=p_s_df['Power Score'],
        marker=dict(color=cs),
        text=p_s_df['Power Score'],
        textposition='outside',
        textfont=dict(size=10)
    )
]

layout = {
    'title': 'Deep Learning Framework Power Scores 2018',
    'xaxis': {'title': 'Framework'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### That's the end! 
### See [this Medium article](https://towardsdatascience.com/deep-learning-framework-power-scores-2018-23607ddf297a) for a discussion of the state of Python deep learning frameworks in 2018 featuring these charts.
## Pleave upvote if you found this interesting or informative!